In [11]:
%pip install --upgrade tensorflow_hub
import tensorflow_hub as hub
import numpy as np
import tensorflow as tf
import tensorflow_text as tftext

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
## loading bert from tensorhub
module_url = "https://tfhub.dev/tensorflow/bert_en_uncased_L-24_H-1024_A-16/1"
bert_layer = hub.KerasLayer(module_url, trainable=False)

2022-11-04 15:19:19.248836: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2022-11-04 15:19:19.248900: W tensorflow/stream_executor/cuda/cuda_driver.cc:269] failed call to cuInit: UNKNOWN ERROR (303)
2022-11-04 15:19:19.248928: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (Mi-Notebook-Ultra): /proc/driver/nvidia/version does not exist
2022-11-04 15:19:19.249916: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
!wget --quiet https://raw.githubusercontent.com/tensorflow/models/master/official/nlp/bert/tokenization.py 


In [9]:
import bert.tokenization as tokenization

vocab_file = bert_layer.resolved_object.vocab_file.asset_path.numpy()
do_lower_case = bert_layer.resolved_object.do_lower_case.numpy()
tokenizer = tokenization.FullTokenizer(vocab_file, do_lower_case)

In [5]:
tokenizer.tokenize('Where are you going ?')

UnparsedFlagAccessError: Trying to access flag --preserve_unused_tokens before flags were parsed.

In [7]:
import pandas as pd

train = pd.read_csv("/home/mdirfan-code/projects/Sentimeter-v1.0/BERT_Model/Datasets/Training_dataset.csv",usecols=['sentiment','tweet_text'])
train

,sentiment,tweet_text
0,Irrelevant,Last Weekend league for Fifa 20 Glad I could f...
1,Positive,omg i'm so excited to watch dk play pubg
2,Negative,all others who have problems with
3,Positive,in
4,Irrelevant,minho felix de jeongin sucked at pubg pretty h...
...,...,...
59191,Negative,you
59192,Neutral,I'm not even going to show a 7 2 loss.
59193,Negative,Fuck this call of duty update..
59194,Positive,I should get up feed my dogs such that way whe...


In [8]:
test = pd.read_csv("/home/mdirfan-code/projects/Sentimeter-v1.0/BERT_Model/Datasets/Testing_dataset.csv",usecols=['sentiment','tweet_text'])
test

,sentiment,tweet_text
0,Irrelevant,Looks to me like he failed to check out the wa...
1,Irrelevant,Wow it takes all sorts of crazy people out the...
2,Neutral,Nvidia Unveils The World s Fastest Gaming Moni...
3,Neutral,Huge radio play here. Reinvention Corporate Ev...
4,Negative,SO I HAPPY WHO ABOUT THIS.
...,...,...
14795,Negative,where is all my Mamba Edition extras? I m bee...
14796,Negative,Sell 700k fifa coins fucking this game
14797,Negative,107 for a four game break and I can't even ev...
14798,Irrelevant,has called me a madman.. I understood right fr...


In [10]:
print(f'BERT vocab is stored at     : {vocab_file}')
print(f'BERT model is case sensitive: {do_lower_case}')

BERT vocab is stored at     : b'/tmp/tfhub_modules/88ac13afec2955fd14396e4582c251841b67429a/assets/vocab.txt'
BERT model is case sensitive: True


In [12]:
def load_vocab(vocab_file):
    """Load a vocabulary file into a list."""
    vocab = []
    with tf.io.gfile.GFile(vocab_file, "r") as reader:
        while True:
            token = reader.readline()
            if not token: break
            token = token.strip()
            vocab.append(token)
    return vocab

vocab = load_vocab(vocab_file)

In [13]:
def create_vocab_table(vocab, num_oov=1):
    """Create a lookup table for a vocabulary"""
    vocab_values = tf.range(tf.size(vocab, out_type=tf.int64), dtype=tf.int64)
    init = tf.lookup.KeyValueTensorInitializer(keys=vocab, values=vocab_values, key_dtype=tf.string, value_dtype=tf.int64)
    vocab_table = tf.lookup.StaticVocabularyTable(init, num_oov, lookup_key_dtype=tf.string)
    return vocab_table

vocab_lookup_table = create_vocab_table(vocab)

In [14]:
tokenizer = tftext.BertTokenizer(
    vocab_lookup_table,
    token_out_type=tf.int64,
    lower_case=do_lower_case
  )

In [15]:
tokenizer.tokenize(["the brown fox jumped over the lazy dog"])

<tf.RaggedTensor [[[1996],
  [2829],
  [4419],
  [5598],
  [2058],
  [1996],
  [13971],
  [3899]]]>